In [1]:
import os
import random
import metpy
import datetime
import s3fs
import copy
import numpy as np
import pandas as pd
import xarray as xr
import tensorflow as tf
import tensorflow_hub as hub
from PIL import Image
from pyresample import geometry, grid
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
from skimage.filters import laplace
from skimage.filters import unsharp_mask
from skimage.transform import resize
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
import scipy.stats
from scipy.signal import wiener

fs = s3fs.S3FileSystem(anon = True)
model = hub.load("https://tfhub.dev/captain-pool/esrgan-tf2/1")
random.seed(42)

2022-12-31 16:38:12.455770: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer.so.6'; dlerror: libnvinfer.so.6: cannot open shared object file: No such file or directory
2022-12-31 16:38:12.455894: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer_plugin.so.6'; dlerror: libnvinfer_plugin.so.6: cannot open shared object file: No such file or directory
2022-12-31 16:38:12.455907: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:30] Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
/sharedData2/nshakoor/.conda/envs/fires/lib/python3.7/site-packages/pyresample/bilinear/__init__.py:49: UserWarning: XArray and/or zarr not found, XArrayBilinearResampler won't be available.
  warnings.warn("XArray and/or zarr not found, XArrayBilinearResampler won't be available.

In [2]:
def find_npy_file(path, file, band):
    """
    Returns the string name of another file from the same time for a given GOES band npy file,
    or None if there is no such file in the given directory.
    
    Parameter path: The path where the returned file is located in
    Precondition: path is a string to a directory relative to the current one, with .npy files
    
    Parameter file: A file from the same time the returned file should have
    Precondition: file is a string with the name of the original file, and includes the path
    
    Parameter band: GOES band the returned file should be from
    Precondition: band is a string of length 2
    """
    for x in os.listdir(path):
        if x[19:21] == band and file[23:37] == x[27:41]:
            return x

In [3]:
def control_img(img):
    height, width = img.shape
    smallimg = resize(img, (round(height/2), round(width/2)))
    control = resize(smallimg, (height, width))
    return control

def laplace_sharpening_img(img):
    height, width = img.shape
    smallimg = resize(img, (round(height/2), round(width/2)))
    blurryimg = resize(smallimg, (height, width))
    laplace_edges = laplace(blurryimg)
    sharpimg = blurryimg + 0.8*laplace_edges
    return sharpimg

def unsharpmask_sharpening_img(img):
    height, width = img.shape
    smallimg = resize(img, (round(height/2), round(width/2)))
    blurryimg = resize(smallimg, (height, width))
    sharpimg = unsharp_mask(blurryimg/blurryimg.max(), radius=1, amount=1)*blurryimg.max()
    return sharpimg

def wiener_sharpening_img(img):
    height, width = img.shape
    smallimg = resize(img, (round(height/2), round(width/2)))
    blurryimg = resize(smallimg, (height, width))
    sharpimg = wiener(blurryimg/blurryimg.max())*blurryimg.max()
    return sharpimg

def update_dict_img(fires, cloud_mask, img07, img14, data1, data2, data3, data4, data5):
    """
    Updates the dictionary with sharpening errors of a given file.
    
    Parameter path: The path to the given file
    Precondition: path is a string
    
    Parameter file: The file to perform laplace sharpening on
    Precondition: file is a string
    
    Parameter data: Dictionary containing sharpening errors
    Precondition: data is a dict
    """
    data07 = control_img(img07)
    data14 = control_img(img14)
    datafires = fire_mask(data07, data14)
    datafires = np.logical_and(datafires, np.logical_not(cloud_mask))
    data1['no_of_fires'].append(np.count_nonzero(datafires))
    data1['dice_score'].append(dice_score(fires, datafires))
    otherstats = other_stats(fires, datafires)
    data1['TPR'].append(otherstats[2])
    data1['PPV'].append(otherstats[3])
    
    data07 = laplace_sharpening_img(img07)
    data14 = laplace_sharpening_img(img14)
    datafires = fire_mask(data07, data14)
    datafires = np.logical_and(datafires, np.logical_not(cloud_mask))
    data2['no_of_fires'].append(np.count_nonzero(datafires))
    data2['dice_score'].append(dice_score(fires, datafires))
    otherstats = other_stats(fires, datafires)
    data2['TPR'].append(otherstats[2])
    data2['PPV'].append(otherstats[3])
    
    data07 = unsharpmask_sharpening_img(img07)
    data14 = unsharpmask_sharpening_img(img14)
    datafires = fire_mask(data07, data14)
    datafires = np.logical_and(datafires, np.logical_not(cloud_mask))
    data3['no_of_fires'].append(np.count_nonzero(datafires))
    data3['dice_score'].append(dice_score(fires, datafires))
    otherstats = other_stats(fires, datafires)
    data3['TPR'].append(otherstats[2])
    data3['PPV'].append(otherstats[3])
    
    data07 = wiener_sharpening_img(img07)
    data14 = wiener_sharpening_img(img14)
    datafires = fire_mask(data07, data14)
    datafires = np.logical_and(datafires, np.logical_not(cloud_mask))
    data4['no_of_fires'].append(np.count_nonzero(datafires))
    data4['dice_score'].append(dice_score(fires, datafires))
    otherstats = other_stats(fires, datafires)
    data4['TPR'].append(otherstats[2])
    data4['PPV'].append(otherstats[3])
    
    data07 = esrgan_sharpening_img(img07)
    data14 = esrgan_sharpening_img(img14)
    datafires = fire_mask(data07, data14)
    datafires = np.logical_and(datafires, np.logical_not(cloud_mask))
    data5['no_of_fires'].append(np.count_nonzero(datafires))
    data5['dice_score'].append(dice_score(fires, datafires))
    otherstats = other_stats(fires, datafires)
    data5['TPR'].append(otherstats[2])
    data5['PPV'].append(otherstats[3])
    
def fire_threshold(img07, img14):
    m = (330-210)/(340-295)
    b = 210 - m*295
    points = []
    
    fire_mask = (img14) < (m*img07 + b)
    points = np.array(fire_mask.nonzero()).T
    
    for point in points:
        save = point[0]
        point[0] = point[1]
        point[1] = 500 - save
        
    return points

def fire_mask(img07, img14):
    m = (330-210)/(340-295)
    b = 210 - m*295
    return (img14) < (m*img07 + b)

def dice_score(x, y):
    numerator = 2*np.count_nonzero(np.logical_and(x, y))
    denominator = np.count_nonzero(x) + np.count_nonzero(y)
    return numerator/denominator

def other_stats(x, y):
    false_positives = np.logical_and(np.logical_not(x), y)
    false_negatives = np.logical_and(x, np.logical_not(y))
    #true positive rate
    TPR = np.count_nonzero(np.logical_and(x, y))/np.count_nonzero(x)
    #positive predictive value
    try:
        PPV = np.count_nonzero(np.logical_and(x, y))/np.count_nonzero(y)
    except ZeroDivisionError:
        PPV = float('NaN')
    return [np.count_nonzero(false_positives), np.count_nonzero(false_negatives), TPR, PPV]

In [17]:
mini = 275
maxi = 400

def load_image(array):
    new_array = 255*((array - mini) / (maxi - mini))
    result = np.zeros((array.shape[0], array.shape[1], 3))
    result[:,:,0]= new_array
    result[:,:,1]= new_array
    result[:,:,2]= new_array
    return result

def preprocess_image(array):
    height, width, depth = array.shape
    result = np.zeros((array.shape[0]//2, array.shape[1]//2, 3))
    hr_image = result[:,:,:]
    smallimg = resize(array, (round(height/2), round(width/2), 3))
    hr_image = smallimg
    
    hr_size = tf.convert_to_tensor(hr_image.shape[:-1])
    hr_image = tf.image.crop_to_bounding_box(hr_image, 0, 0, hr_size[0], hr_size[1])
    hr_image = tf.cast(hr_image, tf.float32)
    return tf.expand_dims(hr_image, 0)

def esrgan_sharpening_img(image):
    """
    Returns dictionary containing errors after using the ESRGAN model.
    
    Parameter path: path of image not including file name
    Precondition: path is a string
    
    Parameter file: file of the image
    Precondition: file is a string of a .npy file
    """
    hr_image = preprocess_image(load_image(image))    
    fake_image = model(hr_image)
    fake_image = tf.squeeze(fake_image)
    fake_image = np.mean(fake_image, axis=2)
    fake_image = resize(fake_image, (image.shape[0], image.shape[1]))
    return (fake_image/255)*(maxi - mini) + mini

# Error

In [18]:
filedates = set()
files = []

for file in os.listdir('../goes_files/npy_files'):
    filedates.add(file[27:41])

for file in os.listdir('../goes_files/clear_sky_mask'):
    if file[23:37] in filedates:
        files.append(file)

In [25]:
path = '../goes_files/npy_files/'
pathCM = '../goes_files/clear_sky_mask/'
fires_initial = []
fires_final_control = {'no_of_fires':[], 'dice_score':[], 'TPR':[], 'PPV':[]}
fires_final_lp = copy.deepcopy(fires_final_control)
fires_final_um = copy.deepcopy(fires_final_control)
fires_final_wr = copy.deepcopy(fires_final_control)
fires_final_ml = copy.deepcopy(fires_final_control)

x = 0

while x < 50:
#     try:
    CM = random.choice(files)
    files.remove(CM)

    file07 = find_npy_file('../goes_files/npy_files', CM, '07')
    file14 = find_npy_file('../goes_files/npy_files', CM, '14')
    img07 = np.load(path + file07)
    img14 = np.load(path + file14)

    cloud_mask = np.load(pathCM + CM)
    cloud_mask = cloud_mask.astype(bool)
    fires = np.logical_and(fire_mask(img07, img14), np.logical_not(cloud_mask))

    if np.count_nonzero(fires) != 0:
        fires_initial.append(np.count_nonzero(fires))
        update_dict_img(fires, cloud_mask, img07, img14, fires_final_control, fires_final_lp, fires_final_um, fires_final_wr, fires_final_ml)

        x += 1

#     except:
#         pass

In [26]:
def results(stat):
    data = [fires_final_control, fires_final_lp, fires_final_um, fires_final_wr, fires_final_ml]
    datanames = ['Control', 'Laplace', 'Unsharp Mask', 'Wiener', 'ESRGAN']
    print(stat)
    for x in range(len(data)):
        print(f'     {datanames[x]} {round(np.nanmean(data[x][stat]), 4)} +/- {round(2*np.nanstd(data[x][stat]), 4)}')

In [27]:
print(round(np.mean(fires_initial), 4))
results('no_of_fires')

17.02
no_of_fires
     Control 16.88 +/- 50.6686
     Laplace 20.68 +/- 54.7773
     Unsharp Mask 19.08 +/- 53.8856
     Wiener 15.82 +/- 49.6382
     ESRGAN 16.54 +/- 48.2464


In [28]:
results('dice_score')

dice_score
     Control 0.7457 +/- 0.4995
     Laplace 0.7745 +/- 0.3195
     Unsharp Mask 0.7809 +/- 0.3964
     Wiener 0.6715 +/- 0.6413
     ESRGAN 0.8499 +/- 0.3775


In [29]:
results('TPR')

TPR
     Control 0.7657 +/- 0.5864
     Laplace 0.8838 +/- 0.3598
     Unsharp Mask 0.8388 +/- 0.4565
     Wiener 0.6953 +/- 0.7156
     ESRGAN 0.8536 +/- 0.435


In [30]:
results('PPV')

PPV
     Control 0.7986 +/- 0.3902
     Laplace 0.7075 +/- 0.3624
     Unsharp Mask 0.7682 +/- 0.3749
     Wiener 0.7788 +/- 0.4135
     ESRGAN 0.8678 +/- 0.3581


In [74]:
scipy.stats.ttest_ind(fires_final_control['dice_score'], fires_final_ml['dice_score'])

Ttest_indResult(statistic=-4.747571072327352, pvalue=2.876945880922905e-06)

In [72]:
scipy.stats.ttest_ind(fires_final_lp['dice_score'], fires_final_ml['dice_score'])

Ttest_indResult(statistic=-4.399930686993859, pvalue=1.392302946504219e-05)

In [73]:
scipy.stats.ttest_ind(fires_final_um['dice_score'], fires_final_ml['dice_score'])

Ttest_indResult(statistic=-4.113522777478705, pvalue=4.737109773734443e-05)

In [17]:
mean = []
std = []
path = '../../GOES_Files/npy_files/'
pathCM = '../../GOES_Files/clear_sky_mask/'
fires_initial = []
fires_final_control = {'no_of_fires':[], 'dice_score':[], 'TPR':[], 'PPV':[]}
fires_final_lp = copy.deepcopy(fires_final_control)
fires_final_um = copy.deepcopy(fires_final_control)
fires_final_wr = copy.deepcopy(fires_final_control)
fires_final_ml = copy.deepcopy(fires_final_control)

x = 0

while x < 20:
    try:
        CM = random.choice(files)
        files.remove(CM)

        file07 = find_npy_file('../../GOES_Files/npy_files', CM, '07')
        file14 = find_npy_file('../../GOES_Files/npy_files', CM, '14')
        img07 = np.load(path + file07)
        img14 = np.load(path + file14)

        cloud_mask = np.load(pathCM + CM)
        cloud_mask = cloud_mask.astype(bool)
        fires = np.logical_and(fire_mask(img07, img14), np.logical_not(cloud_mask))
        
        if np.count_nonzero(fires) != 0:
            fires_initial.append(np.count_nonzero(fires))
            update_dict_img(fires, cloud_mask, img07, img14, fires_final_control, fires_final_lp, fires_final_um, fires_final_wr, fires_final_ml)

            x += 1

    except:
        pass

In [18]:
print(round(np.mean(fires_initial), 4))
results('no_of_fires')
results('dice_score')
results('TPR')
results('PPV')

10.4
no_of_fires
     Control 11.1 +/- 30.9509
     Laplace 13.3 +/- 33.7526
     Unsharp Mask 12.35 +/- 31.9892
     Wiener 10.55 +/- 32.2768
     ESRGAN 10.75 +/- 29.3658
dice_score
     Control 0.7921 +/- 0.4721
     Laplace 0.7746 +/- 0.3952
     Unsharp Mask 0.8003 +/- 0.4678
     Wiener 0.7209 +/- 0.572
     ESRGAN 0.8739 +/- 0.3513
TPR
     Control 0.8226 +/- 0.5167
     Laplace 0.8997 +/- 0.381
     Unsharp Mask 0.8964 +/- 0.4756
     Wiener 0.7349 +/- 0.6319
     ESRGAN 0.8762 +/- 0.4164
PPV
     Control 0.8339 +/- 0.3383
     Laplace 0.716 +/- 0.4405
     Unsharp Mask 0.7865 +/- 0.3784
     Wiener 0.7891 +/- 0.5251
     ESRGAN 0.8829 +/- 0.285
